# VIME Tutorial

### VIME: Extending the Success of Self- and Semi-supervised Learning to Tabular Domain

- Paper: Jinsung Yoon, Yao Zhang, James Jordon, Mihaela van der Schaar, 
  "VIME: Extending the Success of Self- and Semi-supervised Learning to Tabular Domain," 
  Neural Information Processing Systems (NeurIPS), 2020.

- Paper link: TBD

- Last updated Date: October 11th 2020

- Code author: Jinsung Yoon (jsyoon0823@gmail.com)

This notebook describes the user-guide of self- and semi-supervised learning for tabular domain using MNIST database.

### Prerequisite
Clone https://github.com/jsyoon0823/VIME.git to the current directory.

### Necessary packages and functions call

- data_loader: MNIST dataset loading and preprocessing
- supervised_models: supervised learning models (Logistic regression, XGBoost, and Multi-layer Perceptron)

- vime_self: Self-supervised learning part of VIME framework
- vime_semi: Semi-supervised learning part of VIME framework
- vime_utils: Some utility functions for VIME framework

In [1]:
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")
  
from data_loader import load_mnist_data
from supervised_models import logit, xgb_model, mlp

from vime_self import vime_self
from vime_semi import vime_semi
from vime_utils import perf_metric

Using TensorFlow backend.


### Set the parameters and define output

-   label_no: Number of labeled data to be used
-   model_sets: supervised model set (mlp, logit, or xgboost)
-   p_m: corruption probability for self-supervised learning
-   alpha: hyper-parameter to control the weights of feature and mask losses
-   K: number of augmented samples
-   beta: hyperparameter to control supervised and unsupervised loss
-   label_data_rate: ratio of labeled data
-   metric: prediction performance metric (either acc or auc)

In [2]:
# Experimental parameters
label_no = 1000  
model_sets = ['logit','xgboost','mlp']
  
# Hyper-parameters
p_m = 0.3
alpha = 2.0
K = 3
beta = 1.0
label_data_rate = 0.1

# Metric
metric = 'acc'
  
# Define output
results = np.zeros([len(model_sets)+2])  

### Load data

Load original MNIST dataset and preprocess the loaded data.
- Only select the subset of data as the labeled data

In [3]:
# Load data
x_train, y_train, x_unlab, x_test, y_test = load_mnist_data(label_data_rate)
    
# Use subset of labeled data
x_train = x_train[:label_no, :]
y_train = y_train[:label_no, :]  

### Train supervised models

- Train 3 supervised learning models (Logistic regression, XGBoost, MLP)
- Save the performances of each supervised model.

In [4]:
# no, dim = y_train.shape
# vector = np.zeros([no,])
  
# # Convert matrix to vector
# for i in range(dim):
#     idx = np.where(y_train[:, i] == 1)
#     vector[idx] = i
# y_train[0]
# vector

In [26]:
np.random.permutation(len(range(10)))

array([1, 7, 3, 8, 9, 6, 2, 4, 5, 0])

In [4]:
# Logistic regression
y_test_hat = logit(x_train, y_train, x_test)
results[0] = perf_metric(metric, y_test, y_test_hat) 

# XGBoost
y_test_hat = xgb_model(x_train, y_train, x_test)    
results[1] = perf_metric(metric, y_test, y_test_hat)   

# MLP
mlp_parameters = dict()
mlp_parameters['hidden_dim'] = 100
mlp_parameters['epochs'] = 100
mlp_parameters['activation'] = 'relu'
mlp_parameters['batch_size'] = 100
      
y_test_hat = mlp(x_train, y_train, x_test, mlp_parameters)
results[2] = perf_metric(metric, y_test, y_test_hat)

# Report performance
for m_it in range(len(model_sets)):  
    
  model_name = model_sets[m_it]  
    
  print('Supervised Performance, Model Name: ' + model_name + 
        ', Performance: ' + str(results[m_it]))


Restoring model weights from the end of the best epoch
Epoch 00060: early stopping
Supervised Performance, Model Name: logit, Performance: 0.8634
Supervised Performance, Model Name: xgboost, Performance: 0.8622
Supervised Performance, Model Name: mlp, Performance: 0.88


### Train & Test VIME-Self
Train self-supervised part of VIME framework only
- Check the performance of self-supervised part of VIME framework.

In [5]:
# Train VIME-Self
vime_self_parameters = dict()
vime_self_parameters['batch_size'] = 128
vime_self_parameters['epochs'] = 10
vime_self_encoder = vime_self(x_unlab, p_m, alpha, vime_self_parameters)
  
# Save encoder
if not os.path.exists('save_model'):
  os.makedirs('save_model')

file_name = './save_model/encoder_model.h5'
  
vime_self_encoder.save(file_name)  
        
# Test VIME-Self
x_train_hat = vime_self_encoder.predict(x_train)
x_test_hat = vime_self_encoder.predict(x_test)
      
y_test_hat = mlp(x_train_hat, y_train, x_test_hat, mlp_parameters)
results[3] = perf_metric(metric, y_test, y_test_hat)
    
print('VIME-Self Performance: ' + str(results[3]))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
54000/54000 [==============================] - 3s 63us/step - loss: 0.2791 - mask_loss: 0.2265 - feature_loss: 0.0263
Epoch 2/10
54000/54000 [==============================] - 3s 60us/step - loss: 0.2439 - mask_loss: 0.2165 - feature_loss: 0.0137
Epoch 3/10
54000/54000 [==============================] - 3s 61us/step - loss: 0.2358 - mask_loss: 0.2124 - feature_loss: 0.0117
Epoch 4/10
54000/54000 [==============================] - 3s 62us/step - loss: 0.2296 - mask_loss: 0.2080 - feature_loss: 0.0108
Epoch 5/10
54000/54000 [==============================] - 3s 63us/step - loss: 0.2237 - mask_loss: 0.2031 - feature_loss: 0.0103
Epoch 6/10
54000/54000 [==============================] - 3s 62us/step - loss: 0.2180 - mask_loss: 0.1980 - feature_loss: 0.0100
Epoch 7/10
54000/54000 [==============================] - 3s 64us/step - loss: 0.2128 - mask_loss: 0.1932 - feature_loss: 0.0098 1s 

### Train & Test VIME

Train semi-supervised part of VIME framework on top of trained self-supervised encoder
- Check the performance of entire part of VIME framework.

In [5]:
file_name = './save_model/encoder_model.h5'

In [6]:
# Train VIME-Semi
vime_semi_parameters = dict()
vime_semi_parameters['hidden_dim'] = 100
vime_semi_parameters['batch_size'] = 128
vime_semi_parameters['iterations'] = 1000
y_test_hat = vime_semi(x_train, y_train, x_unlab, x_test, 
                       vime_semi_parameters, p_m, K, beta, file_name)

# Test VIME
results[4] = perf_metric(metric, y_test, y_test_hat)
  
print('VIME Performance: '+ str(results[4]))





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Iteration: 0/1000, Current loss: 2.1753
Iteration: 100/1000, Current loss: 0.3285
Iteration: 200/1000, Current loss: 0.2934
Iteration: 300/1000, Current loss: 0.2852

Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from ./save_model/class_model.ckpt
[[6.2057097e-04 5.5199960e-04 9.0249110e-04 ... 9.9375534e-01
  3.6049314e-04 1.2494887e-03]
 [1.2440694e-02 3.4922927e-03 5.0418270e-01 ... 7.5298580e-03
  2.3746537e-02 2.5414338e-03]
 [1.6929401e-03 9.8245412e-01 2.1904709e-03 ... 2.9071406e-03
  1.3097257e-03 2.0062935e-03]
 ...
 [1.4667963e-03 6.8529602e-04 1.7360096e-03 ... 2.0774174e-03
  4.1427058e-03 2.0985965e-02]
 [2.9814471e-03 2.8007785e-03 7.5074797e-04 ... 3.9754543e-03
  3.3698773e-03 1.3703289e-03]
 [2.9263005e-04 1.3824327e-04 8.8916137e-04 ... 1.9054806e-04
  4.7264004e-04 6.9491280e-04]]
VIME Performance: 0.9272


### Report Prediction Performances

- 3 Supervised learning models
- VIME with self-supervised part only
- Entire VIME framework

In [1]:
for m_it in range(len(model_sets)):  
    
  model_name = model_sets[m_it]  
    
  print('Supervised Performance, Model Name: ' + model_name + 
        ', Performance: ' + str(results[m_it]))
    
print('VIME-Self Performance: ' + str(results[m_it+1]))
  
print('VIME Performance: '+ str(results[m_it+2]))

NameError: name 'model_sets' is not defined